# Submission using EfficientNetV2B2 Transfer learning

## Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, datasets

2025-08-04 19:42:01.100046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754336521.328217      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754336521.392062      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### View Dataframe and shape

In [2]:
test_df = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/sample_submission.csv')
test_df.head()

,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0
1,95596b92e5066c5c52466c90b69ff089b39f2737,0
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0
3,2c35657e312966e9294eac6841726ff3a748febf,0
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0


In [3]:
print(test_df.shape)

(57458, 2)


### Add .tif extension

In [4]:
test_df['filenames'] = test_df['id'] + '.tif'

In [5]:
test_df.head()

,id,label,filenames
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5.tif
1,95596b92e5066c5c52466c90b69ff089b39f2737,0,95596b92e5066c5c52466c90b69ff089b39f2737.tif
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0,248e6738860e2ebcf6258cdc1f32f299e0c76914.tif
3,2c35657e312966e9294eac6841726ff3a748febf,0,2c35657e312966e9294eac6841726ff3a748febf.tif
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0,145782eb7caa1c516acbe2eda34d9a3f31c41fd6.tif


### Create test image path and print its shape

In [6]:
test_ref = '/kaggle/input/histopathologic-cancer-detection/test'
print(f'Test Images Shape {len(test_ref)}')

Test Images Shape 51


### Use ImageDataGenerator and create the Test Loader

In [9]:
batch_size = 25
test_datagen = ImageDataGenerator(rescale = 1/255)

test_loader_1 = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = test_ref,
    x_col = 'filenames',
    batch_size = batch_size,
    shuffle = False,
    class_mode = None,
    target_size = (260,260)
)

test_loader_2 = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = test_ref,
    x_col = 'filenames',
    batch_size = batch_size,
    shuffle = False,
    class_mode = None,
    target_size = (96,96)
)

Found 57458 validated image filenames.
Found 57458 validated image filenames.


View the imported model

In [10]:
cnn_model_1 = keras.models.load_model('/kaggle/input/cnn-model-5/Cancer_Detection_cnn_model_5.h5')
cnn_model_1.summary()

I0000 00:00:1754336799.967221      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b2 (Functional)  │ (None, 9, 9, 1408)     │     8,769,374 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1408)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       721,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,655,137 (36.83 MB)

 Trainable params: 9,572,847 (36.52 MB)

 Non-trainable params: 82,288 (321.44 KB)

 Optimizer params: 2 (12.00 B)

In [11]:
cnn_model_2 = keras.models.load_model('/kaggle/input/mwv-final-project-training-model-3/Cancer_Detection_cnn_model_2.h5')
cnn_model_2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet101 (Functional)          │ (None, 3, 3, 2048)     │    42,658,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,930,819 (163.77 MB)

 Trainable params: 42,825,473 (163.37 MB)

 Non-trainable params: 105,344 (411.50 KB)

 Optimizer params: 2 (12.00 B)

### Make a prediction using two of the models created (both transfer learning models that performed the best)

In [12]:
test_pred_1 = cnn_model_1.predict(test_loader_1)
print(test_pred_1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1754336812.869234      78 service.cc:148] XLA service 0x7adf1c0026c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754336812.870196      78 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754336813.808712      78 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/2299 ━━━━━━━━━━━━━━━━━━━━ 6:33:10 10s/step

I0000 00:00:1754336818.810214      78 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2299/2299 ━━━━━━━━━━━━━━━━━━━━ 473s 201ms/step
[[5.7319257e-06]
 [1.2956397e-02]
 [7.4316704e-06]
 ...
 [3.6600217e-05]
 [1.1560665e-03]
 [1.1697331e-03]]


In [13]:
test_pred_2 = cnn_model_2.predict(test_loader_2)
print(test_pred_2)

2299/2299 ━━━━━━━━━━━━━━━━━━━━ 87s 35ms/step
[[0.08565693]
 [0.4679429 ]
 [0.00085811]
 ...
 [0.00121445]
 [0.00472221]
 [0.21866043]]


Print first 10 test predictions

In [14]:
print(test_pred_1[:10].round(2))
print(test_pred_1)

[[0.  ]
 [0.01]
 [0.  ]
 [0.  ]
 [0.03]
 [1.  ]
 [1.  ]
 [0.47]
 [0.  ]
 [0.06]]
[[5.7319257e-06]
 [1.2956397e-02]
 [7.4316704e-06]
 ...
 [3.6600217e-05]
 [1.1560665e-03]
 [1.1697331e-03]]


In [20]:
print(test_pred_2[:10].round(2))

[[0.09]
 [0.47]
 [0.  ]
 [0.03]
 [0.04]
 [0.98]
 [0.78]
 [0.57]
 [0.03]
 [0.84]]


## Combine the scores and divide them. See if the average of the two models together performs better than the best model

In [21]:
ensemble_prediction = (test_pred_1+test_pred_2)/2

## Change labels to the ensemble predictions

In [22]:


cancer_submission = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/sample_submission.csv')
cancer_submission.label = ensemble_prediction
cancer_submission.head(25)

,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0.042831
1,95596b92e5066c5c52466c90b69ff089b39f2737,0.240450
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0.000433
3,2c35657e312966e9294eac6841726ff3a748febf,0.013414
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0.035279
5,725dabe6ecccc68b958a2c7dd75bcbf362c7cb03,0.992197
6,aa0307865281d4484ddf8c637c348292968b93a7,0.890835
7,f4e5dc9c949920f1b3362982e15e99bf6f3ef83b,0.523241
8,95e08c9cedc28a9b4a86f4fc1e06c1972134be08,0.013750
9,ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb,0.448312


In [23]:
cancer_submission.to_csv('cancer_detection_submission_ensemble.csv', index = False, header = True)

In [ ]:
https://www.kaggle.com/code/mattvierheller/mwv-final-project-training-model-5
https://www.kaggle.com/code/mattvierheller/mwv-final-project-training-model-3
